In [29]:
# nltk.download('stopwords')

In [30]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

In [31]:
# Gensim
from gensim.utils import simple_preprocess
from gensim.models import LdaModel, Phrases
from gensim.models.phrases import Phraser
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [32]:
# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt
%matplotlib inline

In [33]:
from nltk.corpus import stopwords
from nltk import Tree

stop_words = stopwords.words('english')

In [34]:
df = pd.read_csv("usnewspaper_china.csv", header=None,)

In [35]:
data = df[0].dropna().tolist()

In [36]:
# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

<>:2: DeprecationWarning: invalid escape sequence \S
<>:5: DeprecationWarning: invalid escape sequence \s
<>:2: DeprecationWarning: invalid escape sequence \S
<>:5: DeprecationWarning: invalid escape sequence \s
/var/folders/fd/qmhdtpdj7kj77ftrqfj87hmc0000gn/T/ipykernel_70879/1419459543.py:2: DeprecationWarning: invalid escape sequence \S
  data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
/var/folders/fd/qmhdtpdj7kj77ftrqfj87hmc0000gn/T/ipykernel_70879/1419459543.py:5: DeprecationWarning: invalid escape sequence \s
  data = [re.sub('\s+', ' ', sent) for sent in data]


In [37]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[0])

['in', 'the', 'new', 'sino', 'russian', 'defense', 'relationship', 'china', 'does', 'what', 'it', 'wants', 'and', 'there', 'isn', 'whole', 'lot', 'russia', 'can', 'do', 'about', 'it', 'xi', 'jingping', 'recent', 'visit', 'to', 'moscow', 'his', 'first', 'since', 'vladimir', 'putin', 'invaded', 'ukraine', 'last', 'year', 'was', 'summed', 'up', 'by', 'historian', 'sergey', 'radchenko', 'in', 'this', 'way', 'the', 'summit', 'can', 'be', 'summarised', 'by', 'the', 'chinese', 'saying', '雷声大雨点小', 'loud', 'thunder', 'but', 'few', 'raindrops', 'scratch', 'that', 'even', 'the', 'thunder', 'wasn', 'all', 'that', 'loud', 'the', 'meeting', 'which', 'apparently', 'produced', 'no', 'major', 'policy', 'shifts', 'nor', 'even', 'notable', 'statements', 'of', 'support', 'did', 'further', 'illustrate', 'tectonic', 'shift', 'in', 'the', 'supposed', 'no', 'limits', 'relationship', 'china', 'is', 'taking', 'the', 'lead', 'in', 'nearly', 'every', 'aspect', 'including', 'in', 'the', 'defense', 'industrial', 's

In [38]:
# Define functions for stopwords and lemmatization
def remove_stopwords(texts):
    return [[word for word in doc if word not in stop_words] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [39]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm')

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_nostops)

In [40]:
# [[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

## Bigrams and Trigrams

In [41]:
bigram = Phrases(data_words, min_count=20, threshold=100) # higher threshold fewer phrases.
trigram = Phrases(bigram[data_words], threshold=100)

bigram_mod = Phraser(bigram)
trigram_mod = Phraser(trigram)

In [42]:
print("Specific Bigram Example:", ' '.join(bigram_mod[data_words[1]]))
                                                      
print("Specific Trigram Example:", ' '.join(trigram_mod[bigram_mod[data_words[0]]]))

Specific Bigram Example: cookie_preferences cookie_list cookie_list cookie is small_piece of data text_file that website when visited by user_asks your_browser to store on your device in order to remember information about you such as your language_preference or login information those cookies are set by us and called first party cookies we also use third party cookies which are cookies from domain_different than the domain of the website you are visiting for our advertising and marketing efforts more specifically we use cookies and other tracking technologies for the following purposes_strictly necessary cookies we do not allow you to opt_out of our certain_cookies as they are necessary to ensure the proper_functioning of our website such as prompting our cookie_banner and remembering_your privacy_choices and or to monitor_site performance these cookies are not used in way that constitutes_sale of your data under the ccpa_you can set your_browser to block or alert you about these cook

In [43]:
data_with_phrases = [trigram_mod[bigram_mod[doc]] for doc in data_words]

In [44]:
# Create Dictionary
id2word = corpora.Dictionary(data_with_phrases)

# Create Corpus
texts = data_with_phrases

# Term Frequency list   
corpus = [id2word.doc2bow(text) for text in texts] 

In [45]:
# Build LDA model
lda_model = LdaModel(corpus=corpus,
                id2word=id2word,
                num_topics=20, 
                random_state=100,
                update_every=1,
                chunksize=100,
                passes=10,
                alpha='auto',
                per_word_topics=True)

In [59]:
# Check the type of the numbers in the LDA topics
for topic in lda_model.print_topics():
    for word, prob in lda_model.show_topic(topic[0]):
        if isinstance(prob, complex):
            print(f"Complex number found in topic {topic[0]} for word '{word}'")

In [60]:
# Print the Keyword
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0, '0.080*"die" + 0.052*"in" + 0.047*"der" + 0.043*"und" + 0.030*"china" + 0.026*"das" + 0.023*"von" + 0.019*"zu" + 0.016*"ist" + 0.016*"fur"'), (1, '0.060*"the" + 0.039*"in" + 0.030*"to" + 0.026*"of" + 0.021*"china" + 0.020*"and" + 0.016*"on" + 0.014*"year" + 0.014*"for" + 0.013*"by"'), (2, '0.024*"ad" + 0.005*"video" + 0.004*"content" + 0.002*"too_loud" + 0.002*"ad_froze" + 0.002*"ad_audio" + 0.002*"finish_loading" + 0.002*"never_loaded" + 0.002*"load_content" + 0.000*"petitions"'), (3, '0.176*"mexico" + 0.049*"new" + 0.035*"canada" + 0.027*"washington" + 0.014*"new_york" + 0.012*"south_carolina" + 0.012*"state" + 0.012*"pacific" + 0.011*"islands" + 0.010*"of"'), (4, '0.125*"the" + 0.057*"of" + 0.047*"and" + 0.037*"in" + 0.032*"to" + 0.013*"for" + 0.013*"on" + 0.012*"from" + 0.009*"by" + 0.009*"as"'), (5, '0.032*"with" + 0.027*"and" + 0.026*"to" + 0.015*"you" + 0.015*"subscribe" + 0.015*"your" + 0.015*"https_yhoo" + 0.014*"news" + 0.013*"it" + 0.011*"get"'), (6, '0.092*"the" + 0.03

In [47]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

In [48]:
try:
    vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
except Exception as e:
    print(f"Error preparing LDA visualization: {e}")

In [49]:
vis

TypeError: Object of type complex is not JSON serializable

PreparedData(topic_coordinates=                        x                   y  topics  cluster       Freq
topic                                                                    
11     0.305614+0.000000j -0.032089+0.000000j       1        1  29.773162
4      0.300359+0.000000j -0.061839+0.000000j       2        1  19.336621
9      0.303330+0.000000j -0.048647+0.000000j       3        1  17.136524
1      0.297468+0.000000j -0.014963+0.000000j       4        1   9.133123
6      0.266709+0.000000j -0.013303+0.000000j       5        1   5.213652
7      0.275285+0.000000j -0.035780+0.000000j       6        1   4.858502
19    -0.028492+0.000000j  0.461540+0.000000j       7        1   4.440817
18     0.231627+0.000000j -0.008444+0.000000j       8        1   4.321904
17     0.149146+0.000000j  0.049226+0.000000j       9        1   2.513920
13     0.151582+0.000000j -0.030139+0.000000j      10        1   1.273949
0     -0.083972+0.000000j  0.189698+0.000000j      11        1   1.116500
8      

In [61]:
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  

coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -10.291464828236148


/Users/javi/anaconda3/lib/python3.9/site-packages/gensim/topic_coherence/direct_confirmation_measure.py:204: RuntimeWarning: divide by zero encountered in scalar divide
  m_lr_i = np.log(numerator / denominator)
/Users/javi/anaconda3/lib/python3.9/site-packages/gensim/topic_coherence/indirect_confirmation_measure.py:323: RuntimeWarning: invalid value encountered in scalar divide
  return cv1.T.dot(cv2)[0, 0] / (_magnitude(cv1) * _magnitude(cv2))



Coherence Score:  nan


In [ ]:
# nlp.add_pipe('sentencizer')

In [ ]:
# create sentences
# sentences = []
# for item in data:
#     doc = (nlp(item))
#     sentence = [sent.text.strip() for sent in doc.sents]
#     sentences.append(sentence)

In [ ]:
# sentences[0][0]

In [ ]:
# # simple preprocess, stopwords, lemma
# data_words = []
# for document in sentences:
#     words = list(sent_to_words(document))
#     data_words_nostops = remove_stopwords(words)
#     data_words.append(lemmatization(data_words_nostops))


# Create tree representation

In [ ]:
data_lemmatized[to_nltk_tree(sent.root).pretty_print() for sent in doc.sents]

In [ ]:
def to_nltk_tree(node):
    if node.n_lefts + node.n_rights > 0:
        return Tree(node.orth_, [to_nltk_tree(child) for child in node.children])
    else:
        return node.orth_

In [ ]:
doc = nlp(data[0])
[to_nltk_tree(sent.root).pretty_print() for sent in doc.sents]

In [ ]:
from IPython.display import HTML
css_str = '<style> \
.jp-Button path { fill: #616161;} \
text.terms { fill: #616161;} \
.jp-icon-warn0 path {fill: var(--jp-warn-color0);} \
.bp3-button-text path { fill: var(--jp-inverse-layout-color3);} \
.jp-icon-brand0 path { fill: var(--jp-brand-color0);} \
text.terms { fill: #616161;} \
</style>'
display(HTML(css_str ))